# Unless you have a massive.com subscription, this will run into 429er errors

In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
bluechips = {
    "AAPL": "apple",
    "JNJ": "johnson_and_johnson",
    "XOM": "exxon_mobil",
    "JPM": "jpmorgan",
    "PG": "procter_and_gamble",
}

In [3]:
import os
os.getcwd()

'/home/lukas/Documents/Github/casml4se-stonkswagen/notebooks/models/baseline_data'

In [5]:
from datetime import datetime
from ml.common.dataRetriever.data_retriever import DataRetriever

#directory in project root.
datapath = os.path.join(os.getenv('DATA_PATH'), 'stocks')
# Create retriever
retriever = DataRetriever()
for key,value in bluechips.items():
    df = retriever.get_data(
        symbol=str(key),
        source="STOCKS",
        start_time=datetime(2023, 11, 17),
        end_time=datetime(2025, 11, 17),
        interval="5m",
    )
    df.to_csv(f"{datapath}/{value}.csv", index=False, sep=";")
    print(f"Saved {df.count()} for {value}")

Fetching AAPL from STOCKS (5m)...
Error fetching data from Polygon.io: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v2/aggs/ticker/AAPL/range/5/minute/1705008300000/2025-11-17?cursor=bGltaXQ9NTAwMCZzb3J0PWFzYw (Caused by ResponseError('too many 429 error responses'))


MaxRetryError: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v2/aggs/ticker/AAPL/range/5/minute/1705008300000/2025-11-17?cursor=bGltaXQ9NTAwMCZzb3J0PWFzYw (Caused by ResponseError('too many 429 error responses'))

In [10]:
from ml.common.folder_s3_sync import upload_folder_to_s3
import time

# Set up parameters
local_folder = datapath
unix_timestamp = int(time.time())
s3_folder_prefix = f'datasources/stocks/raw/{unix_timestamp}/' # folder on s3

# Call the function
upload_folder_to_s3(
    local_folder=local_folder,
    s3_folder_prefix=s3_folder_prefix,
    unix_timestamp=unix_timestamp,
    bucket_name="ost-s3"  # optional, defaults to "ost-s3"
)

Found 5 files to upload
Unix timestamp: 1763487534
Target: ost-s3/datasources/stocks/raw/1763487534/

✓ jpmorgan.csv
✓ johnson_and_johnson.csv
✓ procter_and_gamble.csv
✓ apple.csv
✓ exxon_mobil.csv

Upload Summary:
  Uploaded: 5/5
  Failed: 0/5
  Location: ost-s3/datasources/stocks/raw/1763487534/
  Timestamp: 1763487534 (2025-11-18 18:38:54)
